In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
        .load("abfss://bronze@adlsdatabrickseteproject.dfs.core.windows.net/orders")

In [0]:
df.display()

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed("_rescued_data","rescued_data")

In [0]:
df = df.drop("rescued_data")
df.display()

In [0]:
df = df.withColumn("order_date",to_timestamp(col('order_date')))
df.display()

In [0]:
df = df.withColumn("year",year(col('order_date')))
df.display()

In [0]:
df1 = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

In [0]:
df1 = df1.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

In [0]:
df1 = df1.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

### **Classes - OOP**

In [0]:
class windows:

  def dense_rank(self,df):

    df_dense_rank = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

    return df_dense_rank

  def rank(self,df):

    df_rank = df.withColumn("rank_flag",rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

    return df_rank

  def row_number(self,df):  

    df_row_number = df.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

    return df_row_number

In [0]:
df_new = df

In [0]:
df_new.display()

In [0]:
obj = windows()

In [0]:
df_result = obj.dense_rank(df_new)
df_result.display()

### **Data Writing**

In [0]:
df.write.format("delta").mode("append").save("abfss://silver@adlsdatabrickseteproject.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_ete_project_catalog.silver.orders_silver 
USING DELTA 
LOCATION 'abfss://silver@adlsdatabrickseteproject.dfs.core.windows.net/orders'